# 도커 컨테이너와 파일을 공유하기

도커 컨테이너와 호스트 컴퓨터 간에 파일을 공유해야 할 때는 다음 방법을 사용할 수 있다.

1. 공유 폴더를 이용하는 법
1. `docker cp` 명령을 사용하는 법
1. scp 파일 전송 프로그램을 이용하는 법

## 공유 폴더를 이용하는 법

만약 `docker run` 명령으로 도커 컨테이너를 시작할 때 `-v` 옵션으로 컨테이너-호스트 간의 공유 폴더를 만들었다면 공유 폴더에 공유하고자 하는 폴더를 넣은 뒤 리눅스 `mv` 명령이나 호스트의 파일 복사 명령으로 원하는 위치로 이동하면 된다. 

예를 들어 `test.txt` 이라는 파일을 윈도우즈 호스트에서 도커 컨테이너로 이동하려고 하는 경우를 생각하자. 또 호스트의 사용자 홈 디렉터리가 도커 컨테이너의 `~/hosthome` 이라는 폴더 이름으로 공유되고 있다고 가정하자.

1. `test.txt` 파일을 호스트의 사용자 홈 디렉터리로 복사한다.
1. 도커 컨테이너의 터미널(콘솔)을 열고 도커 컨테이너 안에서 다음과 같이 명령한다.
```
dockeruser@306d4d3d002b:~$ mv ~/hosthome/test.txt ~
```

## `docker cp` 명령을 사용하는 법

이 방법은 공유 폴더를 지정하지 않은 상태에서도 쓸 수 있다. 도커 터미널에서 다음과 같이 명령하면 된다.
```
$ docker cp 컨테이너_이름:컨테이너_폴더나_파일_이름 호스트컴퓨터의_폴더
```

예를 들어 rpython 이란 이름의 컨테이너 내에 있는 /home/dockeruser/notebook 폴더를 통째로 호스트 컴퓨터의 d:/ 라는 폴더 아래로 복사하고 싶으면 다음과 같이 명령한다.
```
$ docker cp rpython:/home/dockeruser/notebook d:/
```

만약 data 폴더 안에 untitled.ipynb 라는 파일만 있었으면 다음과 같이 notebook 폴더가 만들어지고 그 아래에 untitled.ipynb 파일이 있는 것을 볼 수 있다.
```
$ dir d:/notebook/
untitled.ipynb
```

만약 datascienceschool/rpython 이미지를 사용하고 있다면 /home/dockeruser 폴더를 통째로 복사하지 않도록 주의한다. 이 폴더 아래에는 Anaconda 설치 파일도 함께 있어서 필요없이 많은 파이썬 실행 파일과 패키지 파일까지 복사된다.

이 방법의 단점은 아직까지 * 등의 와일드 카드(wild card)를 지원하지 않는다는 점이다. 따라서 정확한 파일이름이나 폴더 이름을 지정해야 한다.

반대로 호스트 컴퓨터에 백업해 놓은 파일이나 폴더를 컨테이너 안으로 복사하려면 다음과 같이 파일 인수를 바꾸면 된다.
```
$ docker cp d:/notebook rpython:/home/dockeruser/
```

다만 이 경우에는 호스트 컴퓨터의 사용자와 컨테이너의 사용자가 다르면 permission 오류가 발생할 수 있으므로 도커 컨테이너 안에서 다음과 같이 폴더 소유자를 변경해 주어야 한다.
```
$ docker attach rpython
dockeruser@bbbd63bfa054:~$ sudo chown dockeruser:dockeruser -R /home/dockeruser/notebook
```

## scp 파일 전송 프로그램을 이용하는 법

만약 도커 컨데이터와 호스트 간에 공유 폴더는 설정되어 있지 않지만 도커 컨테이너 안에서 ssh 서버가 가동 중이라면 scp 파일 전송 프로그램을 이용할 수 있다. `datascienceschool/rpython` 도커 이미지에는 이미 ssh 서버가 가동되도록 설정되어 있기 때문에 scp를 사용할 수 있다.

호스트 컴퓨터가 맥이라면 이미 scp 프로그램이 있을 것이다. 윈도우라면 putty 프로그램 다운로드 웹 사이트에서 윈도우용 scp 프로그램인 pscp 프로그램을 내려받을 수 있다.

* https://www.chiark.greenend.org.uk/~sgtatham/putty/latest.html
  * https://the.earth.li/~sgtatham/putty/latest/w64/pscp.exe


다음으로는 도커 컨데이터에서 가동 중인 ssh 서버의 포트 번호를 알아야 한다. 원래 ssh 서버 포트는 22번이지만 도커 호스트에서도 22번을 사용하기 때문에 보통 다른 포트 번호로 가동한다. 포트 번호는 자동으로 랜덤하게 지정되었을 수도 있고 `docker run` 명령시에 `-p` 옵션으로 지정하였을 수도 있다. 만약 이 글을 순서대로 따라왔다면 아마도 8022 포트를 사용할 것이다. 현재 사용하는 포트 번호는 `docker port` 명령으로 알 수 있다.

이제 다음과 같이 명령한다. 암호를 입력하라는 프롬프트가 나오면 도커 컨테이너 암호를 넣는다. 이 때 다음과 같은 점에 주의한다.

* 윈도우즈에서는 scp 대신 pscp 명령을 사용한다. 
* 포트 번호는 8022이 아닌 경우에는 자신이 사용하는 포트 번호를 사용한다.
* 사용자가 `dockeruser`가 아니면 자신이 사용하는 사용자 아이디와 디렉터리를 사용한다.

```
$ scp -P 8022 test.txt dockeruser@192.168.99.100:/home/dockeruser
dockeruser@192.168.99.100's password:
```
